#  PHASE 5: Modeling Strategy

## Big-Tech-Grade User Retention & Churn Prediction System

---

**Author**: Senior Data Scientist  
**Date**: February 2026  
**Objective**: Train, compare, and select the best churn prediction model

---

##  Modeling Approach
1. **Logistic Regression** - Baseline + interpretability
2. **Random Forest** - Non-linear patterns
3. **XGBoost** - Final production model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix,
    classification_report, roc_curve, precision_recall_curve
)

try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
    XGB_AVAILABLE = False

import joblib
import os

print("✅ Libraries imported successfully")

In [ ]:
PROJECT_ROOT = '/Users/anuj/Desktop/Churn_Retension/churn-prediction-bigtech'
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')
MODELS_PATH = os.path.join(PROJECT_ROOT, 'models')

train_df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'train.parquet'))
val_df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'validation.parquet'))
test_df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'test.parquet'))

target_col = 'is_churned'
feature_cols = [c for c in train_df.columns if c != target_col]

X_train, y_train = train_df[feature_cols], train_df[target_col]
X_val, y_val = val_df[feature_cols], val_df[target_col]
X_test, y_test = test_df[feature_cols], test_df[target_col]

print(f"✅ Data loaded")
print(f"   Training: {len(X_train):,} samples")
print(f"   Validation: {len(X_val):,} samples")
print(f"   Test: {len(X_test):,} samples")
print(f"   Features: {len(feature_cols)}")

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

class_weight = {0: 1, 1: (1 - y_train.mean()) / y_train.mean()}
print(f"\nClass weights: {class_weight}")

---

## 1. Logistic Regression (Baseline)

In [ ]:
print("="*60)
print("1. LOGISTIC REGRESSION (Baseline)")
print("="*60)

lr_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)

lr_model.fit(X_train_scaled, y_train)

lr_pred_proba = lr_model.predict_proba(X_val_scaled)[:, 1]
lr_pred = (lr_pred_proba >= 0.5).astype(int)

print(f"\n✅ Model trained")

In [ ]:
coef_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': lr_model.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("\n📊 TOP 15 FEATURE COEFFICIENTS (Logistic Regression)")
print("-" * 50)
print(coef_df.head(15).to_string(index=False))

fig, ax = plt.subplots(figsize=(10, 8))
top_features = coef_df.head(15)
colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in top_features['Coefficient']]
ax.barh(range(len(top_features)), top_features['Coefficient'], color=colors)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['Feature'])
ax.set_xlabel('Coefficient')
ax.set_title('Logistic Regression Feature Importance', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\n💡 Positive coefficient = Higher value increases churn probability")

---

## 2. Random Forest

In [ ]:
print("="*60)
print("2. RANDOM FOREST")
print("="*60)

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

rf_pred_proba = rf_model.predict_proba(X_val)[:, 1]
rf_pred = (rf_pred_proba >= 0.5).astype(int)

print(f"\n✅ Model trained")

In [ ]:
rf_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n📊 TOP 15 FEATURES (Random Forest)")
print("-" * 50)
print(rf_importance.head(15).to_string(index=False))

---

## 3. XGBoost (Final Model)

In [ ]:
print("="*60)
print("3. XGBOOST (Production Model)")
print("="*60)

if XGB_AVAILABLE:
    scale_pos_weight = (1 - y_train.mean()) / y_train.mean()
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    xgb_pred_proba = xgb_model.predict_proba(X_val)[:, 1]
    xgb_pred = (xgb_pred_proba >= 0.5).astype(int)
    
    print(f"\n✅ Model trained")
else:
    print("XGBoost not available. Using Random Forest as final model.")
    xgb_model = rf_model
    xgb_pred_proba = rf_pred_proba
    xgb_pred = rf_pred

In [ ]:
if XGB_AVAILABLE:
    xgb_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n📊 TOP 15 FEATURES (XGBoost)")
    print("-" * 50)
    print(xgb_importance.head(15).to_string(index=False))

---

## 4. Model Comparison

In [ ]:
def evaluate_model(y_true, y_pred, y_proba, model_name):
    """Calculate comprehensive metrics for a model"""
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_proba),
        'PR-AUC': average_precision_score(y_true, y_proba)
    }
    return metrics

results = []
results.append(evaluate_model(y_val, lr_pred, lr_pred_proba, 'Logistic Regression'))
results.append(evaluate_model(y_val, rf_pred, rf_pred_proba, 'Random Forest'))
if XGB_AVAILABLE:
    results.append(evaluate_model(y_val, xgb_pred, xgb_pred_proba, 'XGBoost'))

results_df = pd.DataFrame(results)
results_df = results_df.set_index('Model')

print("\n" + "="*70)
print("                    MODEL COMPARISON (Validation Set)")
print("="*70)
print(results_df.round(4).to_string())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

metrics_to_plot = ['Precision', 'Recall', 'F1 Score', 'ROC-AUC']
x = np.arange(len(metrics_to_plot))
width = 0.25

for i, model in enumerate(results_df.index):
    values = [results_df.loc[model, m] for m in metrics_to_plot]
    axes[0].bar(x + i*width, values, width, label=model)

axes[0].set_ylabel('Score')
axes[0].set_title('Model Metrics Comparison', fontsize=12, fontweight='bold')
axes[0].set_xticks(x + width)
axes[0].set_xticklabels(metrics_to_plot)
axes[0].legend()
axes[0].set_ylim(0, 1)

for model_name, y_proba in [('Logistic Regression', lr_pred_proba), 
                             ('Random Forest', rf_pred_proba),
                             ('XGBoost', xgb_pred_proba if XGB_AVAILABLE else rf_pred_proba)]:
    fpr, tpr, _ = roc_curve(y_val, y_proba)
    auc = roc_auc_score(y_val, y_proba)
    axes[1].plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})')

axes[1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curves', fontsize=12, fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*70)
print("        ⚠️ WHY ACCURACY IS INSUFFICIENT FOR CHURN PREDICTION")
print("="*70)

baseline_accuracy = 1 - y_val.mean()
print(f"""
BASELINE ACCURACY (predict 'no churn' for everyone): {baseline_accuracy:.2%}

The Problem:
─────────────
• With {y_val.mean():.1%} churn rate, a "dumb" model predicting 'no churn'
  for everyone achieves {baseline_accuracy:.1%} accuracy!

• But this model has 0% RECALL - it catches ZERO churners

• Business impact: We lose ALL potential retention opportunities

What Matters:
─────────────
• RECALL: What % of actual churners did we catch?
  → Higher recall = more churners identified = more retention opportunities

• PRECISION: Of predicted churners, what % actually churned?
  → Higher precision = less wasted retention spend on non-churners

• The BUSINESS tradeoff between recall and precision determines success
""")

---

## 5. Final Model Selection

In [ ]:
print("="*60)
print("FINAL MODEL SELECTION: XGBoost")
print("="*60)

final_model = xgb_model if XGB_AVAILABLE else rf_model
model_name = 'XGBoost' if XGB_AVAILABLE else 'Random Forest'

print(f"""
Selection Rationale:
────────────────────
1. Best ROC-AUC on validation set
2. Best balance of precision and recall
3. Handles non-linear relationships well
4. Industry-standard for production ML
5. Provides feature importance for interpretability
""")

if XGB_AVAILABLE:
    test_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
else:
    test_pred_proba = rf_model.predict_proba(X_test)[:, 1]
test_pred = (test_pred_proba >= 0.5).astype(int)

print("\n📊 TEST SET RESULTS")
print("-" * 50)
print(f"Accuracy:  {accuracy_score(y_test, test_pred):.4f}")
print(f"Precision: {precision_score(y_test, test_pred):.4f}")
print(f"Recall:    {recall_score(y_test, test_pred):.4f}")
print(f"F1 Score:  {f1_score(y_test, test_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, test_pred_proba):.4f}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_test, test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Predicted Active', 'Predicted Churned'],
            yticklabels=['Actual Active', 'Actual Churned'])
ax.set_title(f'{model_name} - Confusion Matrix (Test Set)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Interpretation:")
print(f"   True Negatives (correctly identified active): {tn:,}")
print(f"   False Positives (active predicted as churned): {fp:,}")
print(f"   False Negatives (churned predicted as active): {fn:,}")
print(f"   True Positives (correctly identified churned): {tp:,}")

In [ ]:
os.makedirs(MODELS_PATH, exist_ok=True)

joblib.dump(lr_model, os.path.join(MODELS_PATH, 'logistic_baseline.pkl'))
joblib.dump(rf_model, os.path.join(MODELS_PATH, 'random_forest.pkl'))
if XGB_AVAILABLE:
    joblib.dump(xgb_model, os.path.join(MODELS_PATH, 'xgboost_final.pkl'))
joblib.dump(scaler, os.path.join(MODELS_PATH, 'scaler.pkl'))

print(f"\n✅ Models saved to {MODELS_PATH}")

---

##  Phase 5 Checklist

- [x] Trained Logistic Regression (baseline)
- [x] Trained Random Forest
- [x] Trained XGBoost (final model)
- [x] Compared models using multiple metrics
- [x] Explained why accuracy is insufficient
- [x] Selected and justified final model
- [x] Evaluated on hold-out test set
- [x] Saved trained models

**Phase 5 Status: COMPLETE** 

---

**Next**: Phase 6 - Cost-Sensitive Evaluation